# <span style="color:seagreen; font-family:freestyle script; font-size:4em">LNU till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Hämta variabler från LNU:s kodbok</span>
***

# Innehållsförteckning  
0. [Problem](#0)  
1. [Bibliotek](#1)  
2. [Camelot](#2)  
3. [Kolumner](#3)  
    3.1 [Variabelnamn](#3.1)  
    3.2 [Beskrivning](#3.2)  
    3.3 [Enkättext](#3.3)  
4. [Svarstexter](#4.)  
5. [Finns alla variabler med?](#5.)  
6. [Skapa .csv](#6.)

***

# <a id = "0">0. Problem</a>  

1. Har det hamnat siffror i kolumnen enkätexter? Kanske för variabler som är tagna från register. Ersätt dessa med NaN isåfall.  
2. Finns alla variabler med kanske göra en lista och kolla på ngt sätt?

***  

På [den här sidan](https://github.com/atlanhq/camelot) finns bakgrundsinformation.

# <a id = "1">1. Bibliotek</a>  
Jag behöver pandas, re (för regex) och camelot. Camelot är modulen som hämtar "tabeller" fårn pdf-filer.

In [1]:
import camelot
import pandas as pd
import re
import numpy as np

# <a id = "2">2. Camelot</a>  
Läs in PDF-filen. Just den här filen har tre sidor. Camelot ska lägga "tabellerna" från pdf:en i en lista. Av någon anledning hämtar den bara tabellen från första sidan.

In [2]:
tables = camelot.read_pdf('LNU303.pdf')
tables

<TableList n=1>

Gör om "tabellen" till en DataFrame (`df`). Det verkar som att varje ruta i LNU:s kodbok blir en cell i `df` (som bara har en kolumn). Värdet i cellen är en sträng. Camelot tycks ha plockat varje textbit i rutan och lagt ihop till en lång sträng. Delarna i strängen skiljs åt av "\n".  

In [3]:
tables.n

1

In [4]:
df = tables[0].df
df

,0
0,"48 \nVårdförbundet (SHSTF) \n83 \n3,5 \n49 \nS..."
1,"Z831 \n \nFACKLIG MÖTESAKTIVITET \nX803, jfr Y..."
2,"Z832 \n \nFACKLIGA UPPDRAG \nX804, Y621, \nU65..."


Såhär ser den första cellen ut.

In [5]:
df.loc[0][0]

'48 \nVårdförbundet (SHSTF) \n83 \n3,5 \n49 \nSkolledarförbundet \n5 \n0,2 \n50 \nSkogs- och lantbrukstjänstemannaförbundet (SLF) \n1 \n0,0 \n51 \nTandläkarförbundet (STF) \n5 \n0,2 \n52 \nSveriges universitetslärarförbund (SULF) \n9 \n0,4 \n53 \nTransportarbetareförbundet \n40 \n1,7 \n54 \nSveriges fartygsbefälsförening (SFBF) \n4 \n0,2 \n55 \nSäljarnas riksförbund \n3 \n0,1 \n57 \nTCO, ospec \n6 \n0,3 \n58 \nTeaterförbundet (TF) \n8 \n0,3 \n59 \nTrafik och järnväg (TJ) \n1 \n0,0 \n60 \nTull - kust \n1 \n0,0 \n61 \nVeterinärförbundet (SVF) \n3 \n0,1 \n62 \nLantbrukarnas riksförbund (LRF) \n7 \n0,3 \n63 \nSvenska tecknare \n1 \n0,0 \n65 \nKonstnärernas riksorganisation (KRO) \n1 \n0,0 \n66 \nFackförbund utanför Sverige \n5 \n0,2 \n70 \nArbetsgivar- och företagarorganisati \n20 \n0,8 \n \nTotalt \n2355 \n100 \n88 \nVet ej \n1 \n \n \nEj aktuellt \n2059 \n \n \n \n4415'

Nu skapar jag en lista `L1` som innehåller strängen från varje cell i `df`. Varje sådan sträng har gjorts om till en lista (`L1[i]`) med kortare strängar som tagits fram genom att dela den ursprungliga strängen vid varje "\n". Varje `L1[i]` motsvarar alltså en variabel i LNU:s kodbok.

In [6]:
L1 = [df.loc[i][0].split('\n') for i in df.index]

# <a id = "3">3. Kolumner</a>  
Här förklaras koden som skapar de tre kolumnerna **Variabelnamn**, **Beskrivning** och **Enkättext**. 

## <a id = "3.1">3.1 Variabelnamn</a>   
Den första strängen i varje ruta i kodboken (`L1[i]`) är oftast variabelnamnet.

In [7]:
L1[0][0]

'48 '

Vid sidbrytning kommer dock den första strängen inte se ut sådär. Därför måste jag använda ett `regex`. Alla variabelnamn i LNU 2010 börjar på "Z" och följs av åtminstone en siffra, eller så börjar de med "PZ" och följs av åtminstone en siffra.

In [8]:
regex1 = re.compile(r'^Z\d|^PZ\d')

Listan `namn` ska innehålla alla variabelnamn. Jag går igenom den första strängen i varje cell och ser ifall den är ett "äkta" variabelnamn med hjälp av `regex1`. Är den det läggs strängen i `namn`, i annat fall lägger jag `'sidbrytning'` i listan. På så vis kan jag i ett senare skede ta bort alla rader i tabellen vars variabelnamn är `'sidbrytning'`.

In [9]:
namn = []

for i in range(len(L1)):
    for j in range(len(L1[i])):
        a = len(namn)
        if bool(regex1.findall(L1[i][j])) == True:
            namn.append(L1[i][j])
            break
    if len(namn) == a:
        namn.append('sidbrytning')

## <a id = "3.2">3.2 Beskrivning</a>  
Den andra strängen är alltid (utom vid sidbrytningar så klart) en beskrivning.

In [10]:
L1[1][1]

' '

På samma sätt går det därför att samla alla beskrivningar i en lista. Jag använder en regex som fångar strängar som innehåller åtminstone tre stora bokstäver i följd. Samtliga beskrivningar i LNU:s kodbok är nämligen gjorda med stora bokstäver.

In [11]:
regex2 = re.compile(r'[A-Z]{3}')

Jag går igenom varje sträng i `L1` och plockar ut den första strängen som fångas av `regex2`. Hittar den ingen sådan sträng skriver den 'sidbrytning' istället.

In [12]:
beskrivning = []

for i in range(len(L1)):
    for j in range(len(L1[i])):
        a = len(beskrivning)
        if bool(regex2.findall(L1[i][j])) == True:
            beskrivning.append(L1[i][j])
            break
    if len(beskrivning) == a:
        beskrivning.append('sidbrytning')


    

In [13]:
beskrivning

['Vårdförbundet (SHSTF) ', 'FACKLIG MÖTESAKTIVITET ', 'FACKLIGA UPPDRAG ']

## <a id = "3.3">3.3 Enkättext</a>  
Det är mer komplicerat att hitta enkättexten. Den kommer alltid på tredje, fjärde eller femte plats beroende på hur många "liknande variabler" som listats i högra hörnet i kodboken. För att hantera detta skapar jag en regex som svarar på strängar som beskriver "liknande variabler". Sådana strängar undviks och de övriga läggs i en lista. Enkättexten kan även sträcka sig över flera rader i kodboken. I `L1` blir den därför olika strängar som måste sättas ihop igen. Efter "sista" enkättext-strängen börjar alltid kodlistorna. Eftersom dessa alltid börjar med siffror (oftast 1 men ibland även årtal som 1942) eller "<" kan jag använda ytterligare et regex.

In [14]:
regex3 = re.compile(r'^NY|^jfr|^[XYZUVW]\d|^\s')

regex4 = re.compile(r'^\d|^<')

In [15]:
enkättext = []

for i in range(len(L1)):
    textdelar = []
    for j in range(3, len(L1[i])):
        if bool(regex4.findall(L1[i][j])) == True:
            break
        if bool(regex3.findall(L1[i][j])) == False:
            textdelar.append(L1[i][j])
    enkättext.append(''.join(textdelar))

In [16]:
enkättext

['',
 'Har du varit på något fackföreningsmöte under det senaste året? ',
 'Har  du eller  har  du  haft  någon förtroendepost i  någon fackförening eller facklig sammanslutning (styrelsemedlemskap eller liknande)? ']

Tyvärr verkar det som att vissa variabler saknar enkättext (t.ex. Z3). Dessa är förmodligen väldigt få, det är sanolikt bara vissa bakgrundsvariabler som saknar enkättext. I dessa fall läggs det en siffran från variabelns kodlista i `enkättext`. Detta går enkelt att åtgärna i efterhand genom att ersätta alla siffror med säg `NaN`.

# <a id = "4.">4. Skapa tabellen</a>  
Här kommer all kod ovan samlad i en cell. Anledningen är att vissa sidor i kodboken saknar tabell helt. Detta testas i loopen som går igenom alla sidor i kodboken. Loopen skapar en lista som heter "delar". Den innehåller en dataframe för varje sida i kodboken.

In [17]:
regex1 = re.compile(r'^Z\d|^PZ\d')

regex2 = re.compile(r'[A-Z]{3}')

regex3 = re.compile(r'^NY|^jfr|^[XYZUVW]\d')

regex4 = re.compile(r'^\d|^<')

delar = []

for i in range(32, 358):
    # Läs in sidan i från LNU:s kodbok
    tables = camelot.read_pdf('LNU{}.pdf'.format(i))
    
    print(i)
    # Gör om "tabellen" på sidan till en DataFrame. Vissa sidor, t.ex. 36, har ingen tabell alls. Kolla därför att tabell finns.
    if tables.n != 0:
        df = tables[0].df

        # Dela upp strängen i varje cell vid "\n".
        L1 = [df.loc[i][0].split('\n') for i in df.index]
    
        # Skapa lista med namn (se exempel längre upp för förklaring av regex)
        namn = []

        for j in range(len(L1)):
            for k in range(len(L1[j])):
                a = len(namn)
                if bool(regex1.findall(L1[j][k])) == True:
                    namn.append(L1[j][k])
                    break
            if len(namn) == a:
                namn.append('sidbrytning')
        
        
        # Skapa listan med beskrivningar.         
        beskrivning = []

        for j in range(len(L1)):
            for k in range(len(L1[j])):
                a = len(beskrivning)
                if bool(regex2.findall(L1[j][k])) == True:
                    beskrivning.append(L1[j][k])
                    break
            if len(beskrivning) == a:
                beskrivning.append('sidbrytning')
        
        # Skapa listan med enkättexter.
        enkättext = []

        for j in range(len(L1)):
            textdelar = []
            for k in range(3, len(L1[j])):
                if bool(regex4.findall(L1[j][k])) == True:
                    break
                if bool(regex3.findall(L1[j][k])) == False:
                    textdelar.append(L1[j][k])
            enkättext.append(''.join(textdelar))
        
        # Skapa en dataframe av listorna namn, beskrivning och enkättext. 
        df = pd.DataFrame.from_dict({'Variabelnamn': namn, 'Beskrivning': beskrivning, 'Enkättext': enkättext})
    
        # Nu har hela sidan i blivit en dataframe och läggs i en lista som heter "delar".
        delar.append(df)

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298


Såhär kan en "del" i `delar` se ut. Dessa måste nu slås samman till en stor dataframe.

In [18]:
delar[189]

,Variabelnamn,Beskrivning,Enkättext
0,Z595,VECKOR I EGET FÖRETAG ELLER VERKSAMHET 2009,"& Y370, jfr Antal veckor under 2009: arbetade ..."
1,Z596,VECKOTIMMAR EGET FÖRETAG ELLER VERKSAMHET 2009,"& Y371, jfr Antal timmar per vecka under 2009:..."


Lägg alla DataFrames till en stor. Varje liten representerara en sida i kodboken.

In [19]:
LNU = pd.concat(delar)

Ta bort de rader där variabelnamnet är 'sidbrytning'.

In [20]:
LNU = LNU[LNU.Variabelnamn != 'sidbrytning']

Trots detta finns det två variabler med beskrivningen 'sidbrytning'. Förklaringen är att deras beskrivning är "KÖN". Mitt `regex4` fångar inte upp detta eftersom "Ö" är en svensk bokstsav. 

In [21]:
LNU[LNU['Beskrivning'] == 'sidbrytning']

,Variabelnamn,Beskrivning,Enkättext
3,Z11,sidbrytning,Intervjupersonens kön.
2,PZ2,sidbrytning,Är du man eller kvinna?


Variabeln Z11 ska ha beskrivningen "KÖN" och PZ2 ska ha "P-KÖN".

In [22]:
LNU.loc[LNU.Variabelnamn == 'Z11 ', ['Beskrivning']] = 'KÖN'

LNU.loc[LNU.Variabelnamn == 'PZ2 ', ['Beskrivning']] = 'P-KÖN'

Lägg till en kolumn som säger vilken enkät variabeln kommer ifrån och fyll kolumnen med 'LNU'.

In [23]:
LNU['Enkät'] = 'LNU'

Kolumnen med itemnr innehåller ingen information för LNU-variabler eftersom dessa skapas genom besöksintervju. Lägg därför `NaN` i hela kolumnen. Ändra slutligen ordning en på kolumnerna.

In [24]:
LNU['Itemnr'] = np.nan

LNU = LNU[['Variabelnamn', 'Enkät', 'Beskrivning', 'Itemnr', 'Enkättext']]

Strängarna i `LNU` innehåller whitespace lite överallt i början och slutet. Jag tar bort detta eftersom det kan vara förvirrande när man t.ex. vill hämta en viss variabel.

In [25]:
LNU['Variabelnamn'] = LNU['Variabelnamn'].str.strip()

LNU['Beskrivning'] = LNU['Beskrivning'].str.strip()

LNU['Enkättext'] = LNU['Enkättext'].str.strip()

Gör kolumnen "Variabelnamn" till index. Det blir lättare att välja ut enskilda variabler med LNU.loc.

In [26]:
LNU.set_index('Variabelnamn', inplace = True)

Allt verkar se bra ut.

In [27]:
LNU.head()

,Enkät,Beskrivning,Itemnr,Enkättext
Variabelnamn,,,,
Z2,LNU,PANELKOD,NaN,Intervjupersonens deltagandestatus vid tidigar...
Z3,LNU,INGICK I URVALET FÖR BARN-LNU 2000,NaN,
Z4,LNU,INTERVJUARNUMMER,NaN,Intervjuarens id. Redovisas ej.
Z5,LNU,"INTERVJUDATUM – MÅN, DAG",NaN,"Datum – år, månad och dag – då intervjun genom..."
Z6,LNU,"INTERVJUNS BÖRJAN – TIM, MIN",NaN,Intervjuarens notering av klockslag – timme oc...


# <a id = "5.">5. Finns alla variabler med?</a>  


Jag har strukit över allt i innehållsförteckningen i LNU:s kodbok och lagt i en cell i google sheet. Sedan har jag kopierat denna cell och lagt i en string här.

In [28]:
s1 = 'Z1 UB– NUMMER X1, Y1, U1, V1, W1 Z2 PANELKOD X2 Z3 INGICK I URVALET FÖR BARN-LNU 2000 NY Z4 INTERVJUARNUMMER X4, Y3, U3, V2 Z5 INTERVJUDATUM – MÅN, DAG X5, Y4, U4, jfr V3, jfr W552 Z6 INTERVJUNS BÖRJAN – TIM, MIN X6, Y5, U5, V5 Z7 INTERVJUNS SLUT – TIM, MIN X7, Y6, U6, V708 Z8 TYP AV INTERVJU X10, Y7, U7 Z9 INDIREKT INTERVJU X11, Y8, U8, V719 Z10 INTERVJUSPRÅK X12    KÖN, ÅLDER, YRKE    Z11 KÖN X8, Y10, U10, V718, W2 Z12 FÖDELSEÅR X9, Y11, U11, V815, W533 Z13 SEI – EGEN SYSSELSÄTTNING jfr X14, jfr Y14, jfr U14, jfr V715, jfr W996 Z14 EGEN SOCIOEKONOMISK GRUPP X15, Y13, jfr U13 Z15 NYK80 NUVARANDE YRKE X16 Z16 NYK85 NUVARANDE YRKE X17 Z17 EGP EGEN X18, Y669 Z18 SSYK EGEN NY Z19 ISCO88 EGEN NY Z21 SIOPS EGEN NY    FLYTTNINGAR    Z22 SAMMA BOSTADSORT SEDAN 2000 X19, jfr Y105, jfr U37, jfr V30 Z23 INFLYTTNINGSÅR OCH – MÅNAD, ORTEN X20, jfr Y108, jfr U40, jfr V32, jfr W15 Z24 SKÄL TILL SENASTE FLYTTNING X21, jfr Y107, jfr U39, jfr V33 Z25 INFLYTTNINGSÅR OCH – MÅNAD, BOSTADEN X22, jfr Y111, jfr U42, jfr V35, jfr W96     BOSTADSFÖRHÅLLANDEN    Z26 ANTAL RUM EXKL KÖKET X24, Y113, U45, V38, W71 Z27 BOENDEYTA KVM Z28 ANTAL LÄGENHETER I HUSET X26, Y115, U44, V37, W70 Z29 BÖCKER I BOSTADEN X28, Y117, U65, V55, W88 Z30 DAGLIG TIDNING I BOSTADEN X30 Z31 PROBLEM BOSTAD/OMRÅDE: BULLER X33 Z32 PROBLEM BOSTAD/OMRÅDE: DÅLIGT UNDERHÅLL X34 Z33 PROBLEM BOSTAD/OMRÅDE: OLJUD, GRANNAR X35 Z34 PROBLEM BOSTAD/OMRÅDE: VANDALISM ELLER BROTTSLIGHET X36 Z35 PROBLEM BOSTAD/OMRÅDE: BRIST PÅ KOLLEKTIVA FÄRDMEDEL X37 Z36 PROBLEM BOSTAD/OMRÅDE: RÄDD PÅ KVÄLLEN X40 Z37 BOSTADSFÖRESTÅNDARE X45, Y121, U71, V60, W94 Z38 BOKOSTNAD – EJ FÖRESTÅNDARE X46, Y122, U72, V61 Z39 BESITTNINGSTYP X47, Y123, jfr U73, jfr V62, jfr W95 Z40 HYR I FÖRSTA HAND NY Z41 BOKOSTNAD – HYRA X49, Y125, U75, jfr V63, jfr W98 Z42 UTRYMMESSTANDARD X50, Y130, U68, V745, W626     UTOMLANDSBOENDEN (SEDAN ANKOMST TILL SVERIGE)  Z43 ANTAL UTOMLANDSBOENDEN NY Z44 TID I UTOMLANDSBOENDE NY Z45 ÅR/MÅNAD FÖR START UTOMLANDSBOENDE 1 NY Z46 LAND FÖR UTOMLANDSBOENDE 1 NY Z47 ÅR/MÅNAD FÖR SLUT UTOMLANDSBOENDE 1 NY Z45_a ÅR/MÅNAD FÖR START UTOMLANDSBOENDE 2 NY Z46_a LAND FÖR UTOMLANDSBOENDE 2 NY Z47_a ÅR/MÅNAD FÖR SLUT UTOMLANDSBOENDE 2 NY     UPPVÄXTFÖRHÅLLANDEN      NATIONALITET   Z48 BÅDA FÖRÄLDRARNA FÖDDA I SVERIGE X51 Z49 FÖRÄLDRARNA SVENSKA X52, jfr Y15, jfr U15, jfr V6, jfr W4 Z50 FÖDELSELAND – FADER X53, jfr Y16, jfr U16, jfr V7, jfr W4 Z51 FÖDELSELAND – MODER X54, jfr Y17, jfr U17, jfr V8, jfr W5 Z52 HEMSPRÅK X55, jfr Y18, jfr U18, jfr V9, jfr W6 Z53 IMMIGRATIONSÅLDER X56, jfr Y19, jfr U19, jfr V10, jfr W7 Z54 URSPRUNGSLAND X57 Z55 FÖRMÅGA ATT LÄSA SVENSKA NY Z56 FÖRMÅGA ATT TALA SVENSKA X58 Z57 RELIGIONSTILLHÖRIGHET – FADER jfr X59 Z58 RELIGIONSTILLHÖRIGHET – MODER jfr X60     SYSKON, FÖRÄLDRAR OCH UPPVÄXTFAMILJ  Z59 ANTAL SYSKON, TOTALT X61, jfr Y34, jfr U28, jfr V22, jfr W30 Z60 ANTAL SYSKON I LIVET X62 Z61 ANTAL BRÖDER, TOTALT X63 Z62 ANTAL BRÖDER I LIVET Z63 FÖDELSEÅR SYSKON 1 jfr X65 Z64 FÖDELSEÅR SYSKON 2 jfr X66 Z65 FÖDELSEÅR SYSKON 3 jfr X67 Z66 FÖDELSEÅR SYSKON 4 jfr X68 Z67 FÖDELSEÅR SYSKON 5 jfr X69 Z68 FÖDELSEÅR SYSKON 6 jfr X70 Z69 FÖDELSEÅR SYSKON 7 jfr X71 Z70 FÖDELSEÅR SYSKON 8 jfr X72 Z71 FÖDELSEÅR SYSKON 9 jfr X73 Z72 FÖDELSEÅR SYSKON 10 jfr X74 Z73 FÖDELSEÅR SYSKON 11 jfr X75 Z74 FÖDELSEÅR SYSKON 12 jfr X76 Z75 FÖDELSEÅR SYSKON 13 jfr X77 Z76 FÖDELSEÅR SYSKON 14 jfr X78 Z77 KÖN SYSKON 1 X79 Z78 KÖN SYSKON 2 X80 Z79 KÖN SYSKON 3 X81 Z80 KÖN SYSKON 4 X82 Z81 KÖN SYSKON 5 X83 Z82 KÖN SYSKON 6 X84 Z83 KÖN SYSKON 7 X85 Z84 KÖN SYSKON 8 X86 Z85 KÖN SYSKON 9 X87 Z86 KÖN SYSKON 10 X88 Z87 KÖN SYSKON 11 X89 Z88 KÖN SYSKON 12 X90 Z89 KÖN SYSKON 13 X91 Z90 KÖN SYSKON 14 X92 Z91 POSITION I SYSKONSKARAN X93 Z92 HEL UPPVÄXTFAMILJ jfr X94, jfr Y20, jfr U20, jfr V11, jfr W20 Z93 UMGÄNGESFREKVENS FRÅNLEVANDE FÖRÄLDER X95 Z94 ÅLDER DÅ FÖRÄLDRAR SLUTADE BO TILLSAMMANS NY Z95 BODDE MED STYVFÖRÄLDER NY Z96 BODDE MED STYVSYSKON jfr X114 Z97 TIDPUNKT FÖR FLYTT HEMIFRÅN ÅR/MÅN X115 Z98 KONFLIKTER I UPPVÄXTFAMILJEN X116, Y42, U26, V20, W29 Z99 KONFLIKT BIOLOGISKA FÖRÄLDRAR X117 Z100 KONFLIKT BIOLOGISK OCH STYVFÖRÄLDER X118 Z101 KONFLIKT IP OCH BIOLOGISK MOR X119 Z102 KONFLIKT IP OCH BIOLOGISK FAR X120 Z103 KONFLIKT IP OCH STYVMOR ELLER STYVFAR X121 Z104 KONFLIKT MELLAN ANDRA, VILKA… X122 Z105 FADERNS UTBILDNING X123, jfr Y22, jfr U22, jfr V13, jfr W22 Z106 MODERNS UTBILDNING X124, jfr Y23, jfr U23, jfr V14, jfr W23 Z108 SEI FADERN X126, jfr Y24, jfr U150 Z109 NYK80 FADERN Z110 NYK85 FADERN X128 Z111 SSYK FADERN NY Z112 ISCO88 FADERN NY Z114 SIOPS FADERN NY Z115 FADERNS EGP X848, Y670 Z116 FADERN FÖRETAGARE: ANTAL ANSTÄLLDA X129, jfr Y27, jfr U152, jfr V16, jfr W25 Z117 FADERN JORDBR: AREA X130, Y28, U153, jfr V16, jfr W25 Z118 FADERN FÖRÄLDRALEDIG NY Z120 SEI MODERN X132, jfr Y29, jfr U159 Z121 NYK80 MODERN X133, Y30 Z122 NYK85 MODERN X134 Z123 SSYK MODERN NY Z124 ISCO88 MODERN NY Z126 SIOPS MODERN NY Z127 MODERNS EGP X849, Y671 Z128 MODERN FÖRETAGARE: ANTAL ANSTÄLLDA jfr X135, jfr Y32, jfr U152 Z129 MODERN JORDBR: AREA X136, Y33, jfr U153 Z130 IP:S ÅLDER NÄR MOR BÖRJADE FÖRVÄRVSARBETA X137 Z131 UPPVÄXTFAMILJENS EKONOMI X138, Y36, U25, V19, W28 Z132 FÖRÄLDRAR BOSTADSÄGANDE jfr X140, Y39 Z133 FÖDELSEÅR – MOR X237 Z134 MOR – LEVER X238, jfr Y21, jfr U21, jfr V12, jfr W21 Z135 DÖDSÅR – MOR X239 Z136 CIVILSTÅND – MOR X240 Z137 AVSTÅND – MOR X241 Z138 UMGÄNGE MOR X242 Z139 TELEFONKONTAKT MOR X243 Z140 FÖDELSEÅR – FAR X244 Z141 FAR – LEVER X245, jfr Y21, jfr U21, jfr V12, jfr W21 Z142 DÖDSÅR – FAR X246 Z143 CIVILSTÅND – FAR X247 Z144 AVSTÅND – FAR X248 Z145 UMGÄNGE FAR X249 Z146 TELEFONKONTAKT FAR X250     UPPVÄXTORT  Z147 ANTAL UPPVÄXTORTER X142, jfr Y44, jfr U30, jfr V24, jfr W9 Z148 TYP AV UPPVÄXTORT X143, Y46, U32, V26, W11 Z149 UPPVÄXTLÄN – LAND X144, Y47, U33, V27, W12 Z150 UPPVÄXTKOMMUN X145, Y48, U34, V28, W13     BARNOMSORG OCH SKOLA  Z151 ANTAL MÅNADER PÅ DAGHEM X146, Y51 Z152 VAR HOS DAGMAMMA jfr X147, jfr Y52 Z153 GICK I SEXÅRSVERKSAMHET Z154 CIVILSTÅND ENL INTERVJUN jfr X149, jfr Y57, jfr U90, jfr V68, jfr W36 Z155 ÅR/MÅN NUVARANDE SAMMANBOENDES START X150 Z156 ÅR/MÅNAD FÖR GIFTERMÅL NY Z157 ANTAL PERSONER I HUSHÅLLET X151, Y58, U91, jfr V57, jfr W90 Z158 PARTNER/SAMBO KÖN X152 Z159 PARTNER/SAMBO FÖDELSEÅR X153 Z160 ANTAL HEMMABARN X154 Z161 ANTAL EGNA HEMMABARN X155, jfr Y59, jfr U92, jfr V949, jfr W54 Z162 ANTAL MAKES HEMMABARN X156, jfr Y60, jfr U92, jfr V949, jfr W54 Z163 ANTAL HEMMABARN FÖDDA 1992-2010 X157 Z164 ANTAL YNGRE BARN I HUSHÅLLET FÖDDA 2000-2009 X160, jfr Y81 Z165 ANTAL EGNA HEMMABARN 2009 X161, jfr Y61, jfr U103 Z166 ANTAL EGNA HEMMABARN 2009 FÖDDA 1992-2009 X162 Z167 ANTAL MAKES HEMMABARN 2009 X163, jfr Y62, jfr U103 Z168 ANTAL MAKES HEMMABARN 2009 FÖDDA 1992-2009 X164 Z169 FAMILJETYP X165 Z170 FÖDELSEÅR BARN 1* X166, Y63 Z171 FÖDELSEÅR BARN 2 X167, Y64 Z172 FÖDELSEÅR BARN 3 X168, Y65 Z173 FÖDELSEÅR BARN 4 X169, Y66 Z174 FÖDELSEÅR BARN 5 X170, Y67 Z175 KÖN BARN 1* X171 Z176 KÖN BARN 2 X172 Z177 KÖN BARN 3 X173 Z178 KÖN BARN 4 X174 Z179 KÖN BARN 5 X175 Z180 SAMMANBOR MED FÖRÄLDER X176, Y68 Z181 TREGENERATIONSHUSHÅLL X177, Y69 Z182 GIFT/SAMBOENDE 2009 RESPEKTIVE 2010 X178, Y71, U110 Z183 ANDRA BARN 2009, EJ I HUSHÅLLET 2010 Z184 SYSSELSÄTTNING MAKE/MAKA/SAMBO X181, jfr Y74, jfr U118, jfr V710 Z185 SEI MAKE/MAKA/SAMBO X182, jfr Y72, jfr U117, jfr V711 Z186 NYK80 MAKE/MAKA/SAMBO X183, Y73, U119 Z187 NYK85 MAKE/MAKA/SAMBO X184 Z188 SSYK MAKE/MAKA/SAMBO Z189 ISCO88 MAKE/MAKA/SAMBO NY Z191 SIOPS MAKE/MAKA/SAMBO NY Z192 MAKE/MAKA/SAMBOS EGP X185, Y672 Z193 M EGEN FÖRETAGARE: ANSTÄLLDA X187, jfr Y75, jfr U120 Z194 M ARBETSTID FÖR NÄRVARANDE X189, Y77, U111 Z195 M FULLFÖLJD UTBILDNING Z196 ANTAL YNGRE BARN MED KOMMUNAL BARNTILLSYN X200, jfr Y82 Z197 ANTAL YNGRE BARN MED PRIVAT BARNTILLSYN X201 Z198 ANTAL YNGRE BARN SOM KLARAR SIG SJÄLVA X203, Y85 Z199 ANTAL – I HUSHÅLLET BOENDE – GEMENSAMMA BARN MED NUVARANDE MAKE/MAKA/SAMBO jfr X220 Z200 BIOLOGISKT BARN – BARN 1 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z201 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BARN 1 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z202 DAGAR BOR HOS ANDRA FÖRÄLDERN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z203 TRÄFFAR FRÅNLEVANDE FÖRÄLDERN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z204 IP OCH BARN GÖR NÅGOT MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z205 IP DISKUTERAR MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z206 AVSTÅND TILL ANDRA BIOLOGISKA FÖRÄLDERN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z207 FÖRÄLDRALEDIGHET – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) X221 Z208 FÖRÄLDRALEDIGHET IP – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) X222 Z209 ÅLDER BARNOMSORG – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z210 SJUKDOM/BESVÄR/FUNKTIONSHINDER 1 – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) jfr X231 Z211 STÖD I SKOLAN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z212 TILLSYN – BARN 1 (BARN FÖDDA 1992 ELLER SENARE) NY Z213 HÖGSTA AVSLUTADE UTBILDNING – BARN 1 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z214 PÅGÅENDE UTBILDNING – BARN 1 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z215 SYSSELSÄTTNING – BARN 1 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z216sei SEI BARN 1 NY Z216nyk80 NYK80 BARN 1 NY Z216nyk85 NYK85 BARN 1 NY Z216ssyk SSYK BARN 1 NY Z216isco88 ISCO88 BARN 1 NY Z216siops SIOPS BARN 1 NY Z216egp EGP BARN 1 NY Z217 BIOLOGISKT BARN – BARN 2 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z218 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BARN 2 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z219 DAGAR BOR HOS ANDRA FÖRÄLDERN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z220 TRÄFFAR FRÅNLEVANDE FÖRÄLDERN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z221 IP OCH BARN GÖR NÅGOT MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z222 IP DISKUTERAR MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE Z223 AVSTÅND TILL ANDRA BIOLOGISKA FÖRÄLDERN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z224 FÖRÄLDRALEDIGHET – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) X223 Z225 FÖRÄLDRALEDIGHET IP – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) X224 Z226 ÅLDER BARNOMSORG – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z227 SJUKDOM/BESVÄR/FUNKTIONSHINDER 1 – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) X232 Z228 STÖD I SKOLAN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z229 TILLSYN – BARN 2 (BARN FÖDDA 1992 ELLER SENARE) NY Z230 HÖGSTA AVSLUTADE UTBILDNING – BARN 2 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z231 PÅGÅENDE UTBILDNING – BARN 2 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z232 SYSSELSÄTTNING – BARN 2 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z233sei SEI BARN 2 NY Z233nyk80 NYK80 BARN 2 NY Z233nyk85 NYK85 BARN 2 NY Z233ssyk SSYK BARN 2 NY Z233isco88 ISCO88 BARN 2 NY Z233siops SIOPS BARN 2 NY Z233egp EGP BARN 2 NY Z234 BIOLOGISKT BARN – BARN 3 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z235 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BARN 3 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z236 DAGAR BOR HOS ANDRA FÖRÄLDERN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z237 TRÄFFAR FRÅNLEVANDE FÖRÄLDERN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z238 IP OCH BARN GÖR NÅGOT MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z239 IP DISKUTERAR MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z240 AVSTÅND TILL ANDRA BIOLOGISKA FÖRÄLDERN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z241 FÖRÄLDRALEDIGHET – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) X225 Z242 FÖRÄLDRALEDIGHET IP – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) X226 Z243 ÅLDER BARNOMSORG – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z244 SJUKDOM/BESVÄR/FUNKTIONSHINDER 1 – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) X233 Z245 STÖD I SKOLAN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z246 TILLSYN – BARN 3 (BARN FÖDDA 1992 ELLER SENARE) NY Z247 HÖGSTA AVSLUTADE UTBILDNING – BARN 3 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z248 PÅGÅENDE UTBILDNING – BARN 3 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z249 SYSSELSÄTTNING – BARN 3 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z250sei SEI BARN 3 NY Z250nyk80 NYK80 BARN 3 NY Z250nyk85 NYK85 BARN 3 NY Z250ssyk SSYK BARN 3 NY Z250isco88 ISCO88 BARN 3 NY Z250siops SIOPS BARN 3 NY Z250egp EGP BARN 3 NY Z251 BIOLOGISKT BARN – BARN 4 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z252 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BARN 4 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z253 DAGAR BOR Z254 TRÄFFAR FRÅNLEVANDE FÖRÄLDERN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z255 IP OCH BARN GÖR NÅGOT MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z256 IP DISKUTERAR MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z257 AVSTÅND TILL ANDRA BIOLOGISKA FÖRÄLDERN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z258 FÖRÄLDRALEDIGHET – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) X227 Z259 FÖRÄLDRALEDIGHET IP – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) X228 Z260 ÅLDER BARNOMSORG – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z261 SJUKDOM/BESVÄR/FUNKTIONSHINDER 1 – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) X234 Z262 STÖD I SKOLAN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z263 TILLSYN – BARN 4 (BARN FÖDDA 1992 ELLER SENARE) NY Z264 HÖGSTA AVSLUTADE UTBILDNING – BARN 4 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z265 PÅGÅENDE UTBILDNING – BARN 4 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z266 SYSSELSÄTTNING – BARN 4 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z267sei SEI BARN 4 NY Z267nyk80 NYK80 BARN 4 NY Z267nyk85 NYK85 BARN 4 NY Z267ssyk SSYK BARN 4 NY Z267isco88 ISCO88 BARN 4 NY Z267siops SIOPS BARN 4 NY Z267egp EGP BARN 4 NY Z268 BIOLOGISKT BARN – BARN 5 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z269 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BARN 5 (ALLA BARN OAVSETT FÖDELSEÅR) NY Z270 DAGAR BOR HOS ANDRA FÖRÄLDERN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z271 TRÄFFAR FRÅNLEVANDE FÖRÄLDERN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z272 IP OCH BARN GÖR NÅGOT MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z273 IP DISKUTERAR MED ANDRA BIOLOGISKA FÖRÄLDERN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z274 AVSTÅND TILL ANDRA BIOLOGISKA FÖRÄLDERN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z275 FÖRÄLDRALEDIGHET – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) X229 Z276 FÖRÄLDRALEDIGHET IP – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) X230 Z277 ÅLDER BARNOMSORG – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z278 SJUKDOM/BESVÄR/FUNKTIONSHINDER 1 – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) X235 Z279 STÖD I SKOLAN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z280 TILLSYN – BARN 5 (BARN FÖDDA 1992 ELLER SENARE) NY Z281 HÖGSTA AVSLUTADE UTBILDNING – BARN 5 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z282 PÅGÅENDE UTBILDNING – BARN 5 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z283 SYSSELSÄTTNING – BARN 5 (BARN FÖDDA 1991 ELLER TIDIGARE) NY Z284sei SEI BARN 5 NY Z284nyk80 NYK80 BARN 5 NY Z284nyk85 NYK85 BARN 5 NY Z284ssyk SSYK BARN 5 NY Z284isco88 ISCO88 BARN Z284siops SIOPS BARN 5 NY Z284egp Z285 Z286 EJ HEMMAVARANDE BARN X204, Y89, U124, jfr V954, jfr W59 Z287 ANTAL EJ HEMMAVARANDE BARN X205, Y90, U125, jfr V954, jfr W59 Z288 ANTAL BORTASMÅBARN X206, Y91, U126, jfr V958 Z289 ANTAL BORTASKOLBARN X207, Y92, jfr U127, jfr V958 Z290 ANTAL VUXNA BORTABARN X208, Y93, jfr U129, jfr V956, jfr W61 Z291 ANTAL DÖDA BARN X209, jfr Y94, jfr U130, jfr V957, jfr W62 Z292 ANTAL BORTABARN SOM BOR HOS ANDRA FÖRÄLDERN X210 Z293 KÖN – BORTABARN 1 NY Z294 FÖDELSEÅR – BORTABARN 1 NY Z295 BIOLOGISKT BARN – BORTABARN 1 NY Z296 ANDRA FÖRÄLDERN I IP:S HUSHÅLL – BORTABARN 1 NY Z297 BOR HOS ANDRA BIOLOGISKA FÖRÄLDERN – BORTABARN 1 NY Z298 ÅR SLUTADE BO I SAMMA HUSHÅLL – BORTABARN 1 NY Z299 FÖRÄLDRALEDIGHET – BORTABARN 1  NY Z300 FÖRÄLDRALEDIGHET IP – BORTABARN 1 NY Z301 TRÄFFAR BARN – BORTABARN 1 NY Z302 KONTAKT MED BARN – BORTABARN 1 NY Z303 AVSTÅND TILL BARN – BORTABARN 1 NY Z304 HÖGSTA AVSLUTADE UTBILDNING – BORTABARN 1 NY Z305 PÅGÅENDE UTBILDNING – BORTABARN 1 NY Z306sei SEI BORTABARN 1 NY Z306nyk80 NYK80 BORTABARN 1 NY Z306nyk85 NYK85 BORTABARN 1 NY Z306ssyk SSYK BORTABARN 1 NY Z306isco88 ISCO88 BORTABARN 1 NY Z306siops  NY Z306egp Z307 SÄRBOFÖRHÅLLANDE NY Z308 SÄRBOFÖRHÅLLANDETS LÄNGD NY Z309 SAMMANBOENDE TIDIGARE X190, Y99 14 Z310 ANTAL TIDIGARE SAMMANBOENDEN X191, Y100 Z311 ANTAL TIDIGARE ÄKTENSKAP X192 Z312 ÅR/MÅN FÖR FÖRSTA SAMMANBOENDE X193, jfr Y101 Z313 VARAKTIGHET FÖRSTA SAMMANBOENDE X194, Y102 Z314 CIVILSTÅND FÖRSTA SAMMANBOENDE X195 Z315 ÅR/MÅN SENASTE AVSLUTADE SAMMANBOENDE jfr X196, jfr Y103 Z316 VARAKTIGHET SENASTE AVSLUTADE SAMMANBOENDE X197 Z317 SEPARATIONSANLEDNING SENASTE AVSLUTADE SAMMANBOENDE X198, Y104 Z318 CIVILSTÅND SENASTE AVSLUTADE SAMMANBOENDE X199 Z319 Z320 ANTAL NÄRA VÄNNER X251 Z321 NÄRA VÄNNER – MÄN X253 Z322 NÄRA VÄNNER – FÖDDA I ANNAT LAND X254 Z323 NÄRA VÄNNER – ARBETSLÖSA X256 Z324 TRÄFFAR NÄRA VÄNNER, HUR OFTA X259 Z325 TELEFONKONTAKT MED NÄRA VÄNNER, HUR OFTA X260 Z326 SEI NÄRMASTE VÄN NY Z327 NYK80 NÄRMASTE VÄN NY Z328 NYK85 NÄRMASTE VÄN NY Z329 SSYK NÄRMASTE VÄN NY Z330 ISCO88 NÄRMASTE VÄN NY Z331 SIOPS NÄRMASTE VÄN NY Z332 EGP NÄRMASTE VÄN NY     UTBILDNING    Z342 IP:S UTBILDNINGSÅR X261, Y131, U137, V229, W538 Z343 IP:S SAMMANLAGDA YRKESUTBILDNING jfr Y133, U139, V231, jfr W833 Z344 HÖGSTA AVSLUTADE UTBILDNING I SVERIGE NY Z345 LAND FÖR HÖGSTA UTBILDNING  NY Z346 UTBILDNINGSÅR UTOMLANDS Y157 Z347 LAND FÖR UTLÄNDSK UTBILDNING (dock ej nödvändigtvis högsta avslutade) NY Z348 HÖGSTA UTBILDNINGSNIVÅ UTOMLANDS Y158 Z349 UTBILDNINGSMÅNADER I SVERIGE jfr Y159 Z350 NIVÅ HÖGSTA UTBILDNING  NY Z352 IP:S UTBILDNINGSNIVÅ jfr X305, jfr Y164, jfr U138, jfr V230, jfr W537 Z354 ÅR/MÅNAD AVSLUTAD HÖGSTA UTBILDNING jfr Y166 Z355 STUDERAR NU: TYP AV STUDIER jfr Y168, jfr U568, jfr V596, jfr W368     SYSSELSÄTTNINGSBIOGRAFI    Z359 ANTAL ÅR I FÖRVÄRVSARBETE X311, Y183, U478, V517, W293 Z360 ARBETAT 6 MÅNADER Z361 LAND FÖR FÖRSTA ARBETE OM 6 MÅNADER NY Z362 ÅR/MÅNAD START FÖR FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z363 ÅR/MÅNAD SLUT FÖR FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z364 ÅR/MÅNAD START FÖR SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z365 ÅR/MÅNAD SLUT FÖR SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z366 LAND FÖR SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z367 BÖRJADE ARBETA (I SVERIGE): ÅRTAL X313, Y184, jfr U144 Z368 KVAR I FÖRSTA ARBETET (OM MINST 6 MÅNADER) X314, jfr Y185 Z369 TID I FÖRVÄRVSARBETE, MÅNADER X315, Y186, jfr U478 Z370 I FÖRVÄRVSARBETE SENAST NÄR X316, Y187 Z371 TID I ARBETSLÖSHET, MÅNADER Y188 Z372 ARBETSLÖS MINST 1 MÅNAD SENAST NÄR, ÅRTAL Y191 Z373 ANTAL GÅNGER ARBETSLÖS MINST 1 MÅNAD Y192 Z374 ANSTÄLLNINGSÅR, NUVARANDE ARBETE X322, jfr Y193, jfr U350, jfr V400, jfr W229 Z375 SENIORITET, NUVARANDE ARBETSPLATS, MÅNADER X323, Y195 Z376 SENIORITET, NUVARANDE ARBETE, MÅNADER X324, Y196 Z377 ANTAL FÖRVÄRVSARBETEN, TOTALT X325, Y197 Z378 ANTAL ARBETEN SOM ANSTÄLLD X326, Y198 Z379 ANTAL ARBETSSTÄLLEN SOM ANSTÄLLD Y201 Z380 TID BORTA FRÅN FÖRVÄRVSARBETE, MÅNADER X329, Y202 Z381 TID FÖRÄLDRALEDIGHET, MÅNADER Y203 Z382 TID I HEMARBETE, MÅNADER Y204 Z383 TID I STUDIER, MÅNADER Y205 Z384 ANTAL GGR I STUDIER, MINST 1 MÅNAD Y206 Z385 ANTAL GGR BORTA FRÅN ARBETET, MINST 1 MÅNAD Y207 Z386 ARBETSLIVSAVBROTT SENAST NÄR Y208 Z387 SEI 25 ÅR X336, Y209 Z388 NYK80 25 ÅR Y210 Z389 NYK85 25 ÅR NY Z394 EGP 25 ÅR Y211 Z395 SEKTOR 25 ÅR Y212 Z396 AKTIVITETSTYP 25 ÅR X341, Y213 Z397 ÅLDER VID SENASTE ARBETE 25 ÅR X342, Y214 Z398 ARBETSPLATSENS STORLEK 25 ÅR NY Z399 SEI 30 ÅR X347, Y215 Z400 NYK80 30 ÅR Y216 Z401 NYK85 30 ÅR NY Z406 EGP 30 ÅR Y217 Z407 SEKTOR 30 ÅR Y218 Z408 AKTIVITETSTYP 30 ÅR X352, Y219 Z409 ÅLDER VID SENASTE ARBETE 30 ÅR X353, Y220 Z410 ARBETSPLATSENS STORLEK 30 ÅR NY Z411 SEI 35 ÅR X358, Y221 Z412 NYK80 35 ÅR Y222 Z413 NYK85 35 ÅR NY Z418 EGP 35 ÅR Z419 SEKTOR 35 ÅR Y224 Z420 AKTIVITETSTYP 35 ÅR X363, Y225 Z421 ÅLDER VID SENASTE ARBETE 35 ÅR X364, Y226 Z422 ARBETSPLATSENS STORLEK 35 ÅR NY Z423 SEI 40 ÅR X369, Y227 Z424 NYK80 40 ÅR Y228 Z425 NYK85 40 ÅR NY Z430 EGP 40 ÅR Y229 Z431 SEKTOR 40 ÅR Y230 Z432 AKTIVITETSTYP 40 ÅR X374, Y231 Z433 ÅLDER VID SENASTE ARBETE 40 ÅR X375, Y232 Z434 ARBETSPLATSENS STORLEK 40 ÅR NY Z435 SEI 45 ÅR X380, Y233 Z436 NYK80 45 ÅR Y234 Z437 NYK85 45 ÅR NY Z442 EGP 45 ÅR Y235 Z443 SEKTOR 45 ÅR Y236 Z444 AKTIVITETSTYP 45 ÅR X385, Y237 Z445 ÅLDER VID SENASTE ARBETE 45 ÅR X386, Y238 Z446 ARBETSPLATSENS STORLEK 45 ÅR NY Z447 SEI 50 ÅR X391, Y239 Z448 NYK80 50 ÅR Y240 Z449 NYK85 50 ÅR NY Z454 EGP 50 ÅR Y241 Z455 SEKTOR 50 ÅR Y242 Z456 AKTIVITETSTYP 50 ÅR X396, Y243 Z457 ÅLDER VID SENASTE ARBETE 50 ÅR X397, Y244 Z458 ARBETSPLATSENS STORLEK 50 ÅR NY Z459 SEI FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z460 NYK80 FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z461 NYK85 FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z462 EGP FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z463 FÖRETAGSSTORLEK (EGEN FÖRETAGARE) FÖRSTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z464 SEI SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z465 NYK80 SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z466 NYK85 SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z467 EGP SISTA ARBETE I ANNAT LAND ÄN SVERIGE NY Z468 FÖRETAGSSTORLEK (EGEN FÖRETAGARE) VID SISTA ARBETET I ANNAT LAND ÄN SVERIGE NY Z469 SEI FÖRSTA ARBETE X402, Y245 Z470 NYK80 FÖRSTA ARBETE X403, Y246 Z471 NYK85 FÖRSTA ARBETE  X404 Z476 EGP FÖRSTA ARBETE X405, Y247 Z479 ARBETSPLATSSTORLEK FÖRSTA ARBETE jfr Y249 Z480 SEKTORTILLHÖRIGHET FÖRSTA ARBETE Y250 Z481 SEI NUVARANDE ARBETE X412, Y251 Z482 NYK80 NUVARANDE Z483 NYK85 NUVARANDE ARBETE X414 Z484 SSYK NUVARANDE ARBETE NY Z488 EGP NUVARANDE ARBETE X415, Y253 Z489 SNI NUVARANDE ARBETE NY Z491 ARBETSPLATSSTORLEK NUVARANDE ARBETE X417, jfr Y255 Z492 SEKTORTILLHÖRIGHET NUVARANDE ARBETE X418, Y256 Z493 SYSSELSÄTTNING VID NUVARANDE SAMMANBOENDES START NY Z494 SEI VID NUVARANDE SAMMANBOENDES START NY Z495 NYK80 VID NUVARANDE SAMMANBOENDES START NY Z496 NYK85 VID NUVARANDE SAMMANBOENDES START NY Z501 EGP VID NUVARANDE SAMMANBOENDES START NY Z504 ARBETSPLATSSTORLEK VID NUVARANDE SAMMANBOENDES START NY Z505 SEKTORTILLHÖRIGHET VID NUVARANDE SAMMANBOENDES START NY     HÄLSA OCH VÅRDKONSUMTION    Z506 ALLMÄNT HÄLSOTILLSTÅND X435, Y257 Z507 PROMENERA 100 M X436, Y258, U160, V235, W101 Z508 SPRINGA 100 M X437, Y259, U161, V236, W102 Z509 GÅ I TRAPPOR X438, Y260, U162, V237, W103 Z510 VRIDA VATTENKRAN X439, Y261 Z511 LÄNGD I CM X440, Y262 Z512 VIKT I KG X441, Y263     SJUKDOMAR OCH SYMPTOM DE SENASTE 12 MÅNADERNA  Z513 HUVUDVÄRK X452, Y277, U190, V257, W120 Z514 FÖRKYLNING X453, Y278, U191, V258, W121 Z515 SYNBESVÄR X454, Y279, U192, V259, W122 Z516 HÖRSELNEDSÄTTNING X455, Y280, U193, V260, W123 Z517 BRÖSTVÄRK X456, Y281, U194, V261, W124 Z518 LUFTRÖRSKATARR X457, Y282, U195, V262, W125 Z519 STRUMA X458, Y283, U196, V263, W126 Z520 TUBERKULOS X459, Y284, U197, V264, W127 Z521 VÄRK I SKULDROR/AXLAR X460, Y285, U198, V265, W128 Z522 HJÄRTINFARKT X461, Y286, U199, V266, W129 Z523 STROKE  X462 Z524 HJÄRTSVAGHET X463, Y287, U200, V267, W130 Z525 HÖGT BLODTRYCK X464, Y288, U201, V268, W131 Z526 MAGONT X465, Y289, U202, V269, W132 Z527 MAGSÅR X466, Y290, U203, V270, W133 Z528 VÄRK I RYGG/HÖFTER X467, Y291, U204, V271, W134 Z529 GALLBESVÄR X468, Y292, U205, V272, W135 Z530 NJURBESVÄR X469, Y293, U206, V273, W136 Z531 HEMORROJDER X470, Y294, U207, V274, W137 Z532 URINERINGSBESVÄR X471, Y295, U208, V275, W138 Z533 MENSTRUATIONSBESVÄRZ534 GRAVIDITETSKOMPLIKATION X473, Y297, jfr U210, jfr V278, jfr W141 Z535 UNDERLIVSBESVÄR X474, Y298, U211, V277, W140 Z536 LJUMSKBRÅCK X475, Y299, U212, V279, W142 Z537 ÅDERBRÅCK X476, Y300, U213, V280, W143 Z538 BENSVULLNAD X477, Y301, U214, V281, W144 Z539 LEDVÄRK X478, Y302, U215, V282, W145 Z540 ALLMÄN TRÖTTHET X479, Y303, U216, V283, W146 Z541 SÖMNBESVÄR X480, Y304, U217, V284, W147 Z542 NERVÖSA BESVÄR X481, Y305, U218, V285, W148 Z543 DEPRESSIONER X482, Y306, U219, V286, W149 Z544 PSYKISK SJUKDOM X483, Y307, U220, V287, W150 Z545 SVETTNINGAR X484, Y308, U221, V288, W151 Z546 HOSTA X485, Y309, U222, V289, W152 Z547 ANDNÖD X486, Y310, U223, V290, W153 Z548 YRSEL X487, Y311, U224, V291, W154 Z549 ILLAMÅENDE X488, Y312, U225, V292, W155 Z550 AVMAGRING X489, Y313, U226,V293, W156 Z551 KRÄKNINGAR X490, Y314, U227, V294, W157 Z552 DIARRÉ X491, Y315, U228, V295, W158 Z553 FÖRSTOPPNING X492, Y316, U229, V296, W159 Z554 ÖVERANSTRÄNGNING X493, Y317, U230, V297, W160 Z555 UTSLAG X494, Y318, U231, V298, W161 Z556 CANCER X495, Y319, U232, V299, W162 Z557 BLODBRIST X496, Y320, U233, V300, W163 Z558 SOCKERSJUKA X497, Y321, U234, V301, W164 Z559 ÖVERVIKT X498, Y322, U235, V302, W165 Z560 ORGANISK NERVSJUKDOM X499, Y323, U236, V303, W166 Z561 BESVÄR EFTER SKADA X500, Y324, jfr U189 Z562 ALLERGI X501, Y325 Z563 URINVÄGSINFEKTION X502, Y326 Z564 ÖVRIGA SJUKDOMAR jfr Y327, jfr Y328, jfr U237, jfr U238, jfr V304, jfr V305     VÅRDKONSUMTION  Z565 INLAGD PÅ SJUKHUS X509, Y329, U253, V310, jfr W172 Z566 SÖKT LÄKARE jfr X511, jfr Y331, jfr U255, V321, jfr W176 Z567 SKÖTERSKEBESÖK X515, Y333, U257, V323, W177 Z568 TANDTILLSTÅND X519, Y336, jfr U269, jfr V336, jfr W185     ALKOHOL OCH TOBAK  Z569 RÖKNING X522, Y340, jfr U278, jfr V340, jfr W202, W203 Z570 RÖKT I ANTAL ÅR X523, Y341, U279 Z571 DRICKER VIN, SPRIT MM X525, Y343, U280, jfr V341, jfr W204 Z572 ANTAL GGR DRICKER VIN, SPRIT MM X526 Z573 ANTAL GLAS FYSISK OCH PSYKISK HÄLSA  Z574 RÖRLIGHET X528, Y346, U239, V734, W559 Z575 PSYK VÄLBEF INDEX X529, Y347, U241, V736 Z576 PSYKISKT VÄLBEFINNANDE X530, Y348, U242, V737, W573 Z577 OLLES PSYKINDEX X531, Y349 Z578 VÄRK I RÖRELSEAPPARAT X532, Y350, U243, V739, W567 Z579 OLLES VÄRKINDEX X533, Y351 Z580 CIRKULATIONSSYSTEM X534, Y352, U244, V740, W566 Z581 OLLES CIRKULATIONSINDEX X535, Y353 Z582 MAGBESVÄR X536, Y354, U245, V741, W568 Z583 OLLES MAGINDEX X537, Y355 Z584 AVFÖRINGSBESVÄR X538, Y356, U246, V742, W569 Z585 SUMMERAD SYMPTOMSKALA X539, Y357, U247, V743, jfr W637 Z586 SUMMERAD SYMPTOMSKALA 2 X540, Y358 Z587 ANTAL JA PÅ SYMPTOM X542, Y359, U248, V744, jfr W638 Z588 ANTAL JA PÅ SYMPTOM 2 X543     SYSSELSÄTTNINGSFÖRHÅLLANDEN      FÖRRA ÅRET  Z589 TOTALT ANTAL VECKOR I ARBETE 2009 NY Z590 ÅRSARBETSTIMMAR 2009 jfr X544, jfr Y360, jfr U283, jfr V789, jfr W940 Z591 ANSTÄLLD 2009 jfr X545 & X548, jfr Y361 & Y364, jfr U284 & U288, jfr V342 & V346, jfr W205 & W207, jfr W206 & W208 Z592 VECKOR I ANSTÄLLNING 2009 jfr X546 & X549, jfrY362 & Y366, jfr U285 & U290, jfr V343 & V348, jfr W205 & W207, jfr W206 & W208 Z593 VECKOTIMMAR ANSTÄLLD 2009 jfr X547 & X550, jfr Y363 & Y365, jfrU286 & U289, jfr V344 & V347, jfr W205 & W207, jfr W206 & W208 Z594 ARBETE I FÖRETAG ELLER VERKSAMHET 2009 jfr X557, jfr Y375, jfr U304, jfr V364, jfr W214, jfrW215 Z595 VECKOR I EGET FTG EL VERKSAMH 2009 jfr X558 & X552, jfr Y376 & Y370, jfr U305 & U297, jfr V365 & V357, jfr W214 & W210, jfr W215 & W211 Z596 VECKOTIMMAR EGET FTG EL VERKSAMH 2009    jfr X559 & X553, jfr Y377 & Y371, jfr U306 & U298, jfr V366 & V358, jfr W214 & W210, jfr W215 & W211 Z597 ARBETSLÖS UNDER 2009 jfr X566, jfr Y384, jfr U316, jfr V376, jfr W220 Z598 ARBETSLÖSHETSVECKOR 2009 jfr X567, jfr Y385, jfr U317, jfr V377, jfr W220     FÖRRA VECKAN  Z599 FÖRRA VECKAN: HELTID X570, Y398, jfr U332, jfr V345 Z600 FÖRRA VECKAN: DELTID X571, Y399, jfr U333, jfr V349 Z601 FÖRRA VECKAN: FÖRETAG ELLER JORDBRUK jfr X574 & X572, jfr Y402 & Y400, jfr U338 & U336, jfr V367 & V359 Z602 FÖRRA VECKAN: MEDHJÄLPARE FÖRETAG / JORDBRUK jfr X575 & X573, jfr Y403 & Y401, jfr U339 & U337, jfr V371 & V363 Z603 FÖRRA VECKAN: FRILANS ELLER BISYSSLA X576, Y404, U340, V375 Z604 FÖRRA VECKAN: SÖKTE ARBETE Z605 FÖRRA VECKAN: PENSION X578, Y406, U344, V387 Z606 FÖRRA VECKAN: STUDERADE X580, Y408, U346, V393 Z607 FÖRRA VECKAN: ÖVRIGT X582, Y410, U347, V396     ANSTÄLLDAS ARBETSFÖRHÅLLANDEN    Z608 SEI ANSTÄLLDA X583, Y411, jfr U355, jfr V712 Z609 NYK80 ANSTÄLLDA X584, Y412, jfr U356 Z610 NYK85 ANSTÄLLDA X585 Z611 SSYK ANSTÄLLDA NY Z612 ISCO88 ANSTÄLLDA NY Z614 SIOPS ANSTÄLLDA NY Z615 ANTAL ÅR I SAMMA TYP AV ARBETE NY Z616 ANSTÄLLNINGSFORM jfr X419 Z617 FRÅNVARO SENASTE 3 MÅNADERNA NY Z618 FRÅNVAROANLEDNING. SENASTE 3 MÅN  NY Z619 AVANCEMANGSMÖJLIGHETER, INTERNA jfr X587 & X588 Z620 AVANCEMANGSMÖJLIGHETER, EXTERNA  jfr X587 & X588 Z621 ANSTÄLLNINGSÅR, NUVARANDE ARB  jfr Y193, jfr U350, jfr V400, jfr W229 Z622 ANTAL UNDERSTÄLLDA   X590, Y414, jfr U357, jfr V404, jfr W233  Z623 ARBETSLEDNING BLAND UNDERSTÄLLDA NY Z624 UTBILDNINGSKRAV  X591, Y415, U358, V405, jfr W234 Z625 SKOLÅR UTÖVER FOLK-/GRUNDSKOLA  X592, Y416, U359, V406, jfr W234 Z626 UPPLÄRNINGSTID X593, Y417 Z627 NYTTA AV TIDIGARE KUNSKAP X594, Y418 Z628 SPECIFIK KOMPETENS X596, Y419     ARBETSTIDER FÖREGÅENDE VECKA   Z629 ARBETSTID ELLER SKIFT jfr X601, Y422, jfr U360 Z630 OBEKVÄM ARBETSTID NY Z631 ORDINARIE VECKOARBETSTID X602, Y460, U418, V459, W251 Z632 PASSAR ARBETSTIDEN X603, Y461, U420, V468 Z633 HUR OFTA ÖVERTID X604 Z634 ERSÄTTNING FÖR ÖVERTID X609 Z635 ÖVERNATTNING UTANFÖR HEMMET PGA ARBETE: ANTAL NÄTTER/ÅR X627 Z636 RESTID MINUTER PER DAG X651, jfr Y441, jfr U387, jfr V765, jfr W244, W636     LÖN  Z637 FAST MÅNADSLÖN FÖRE SKATT X611, Y464, U434, V720, jfr W272, W275 Z638 VECKOLÖN FÖRE SKATT X612, Y465, U435, V721, jfr W272, W275 Z639 FAST TIMLÖN X613, Y466, U436, V722, jfr W272, W275 Z640 INDIVIDUELLT ACKORD X614, Y467, U437, jfr V723, V724, jfr W272, W275 Z641 GRUPPACKORD X615, Y468, U438, V725, jfr W272, W275 Z642 MÅNADSLÖN M BONUS/PROVISION Z643 ERSÄTTN FÖR OBEKVÄM ARBETSTID X617, Y470, U441, V729, jfr W272 Z644 ANNAN LÖNEFORM KR/MÅN X618, Y471, U442, V730, jfr W272 Z645 BRUTTOTIMLÖN X619, jfr Y472, jfr U443, jfr V727, jfr W650 Z646 MÅNADSLÖN EFTER SKATT X620, Y473, U433, V490, jfr W276      AUTONOMI OCH INFLYTANDE  Z647 NOGA MED TIDERNA X628, Y482, U422, V470, W254 Z648 STÄMPELUR X629, Y483, U423, V471, W255 Z649 FLEXIBEL ARBETSTID X630, Y484, U424 Z650 GRAD AV FLEXIBEL ARBETSTID NY Z651 KAN GÅ ÄRENDE X631, Y485, U427, V474, W258 Z652 KAN BESTÄMMA ARBETSTAKT X633, Y487, U429 Z653 INFLYTANDE VILKA ARBETSUPPGIFTER X634, Y489 Z654 INFLYTANDE ARBETSMETODER X635, Y490 Z655 LÄR NYA SAKER X636, Y491 Z656 FÅR STÖD AV KAMRATER X638, Y494 Z657 CHEFENS ANTAL UNDERSTÄLLDA   X645 Z658 CHEFENS KÖN X646 Z659 CHEFEN UTLANDSFÖDD X647 Z660 CHEFEN ÄR HÖGSTA CHEF PÅ ARBETSPLATSEN  NY Z661 IP:S ARBETE SVÅRT ATT KONTROLLERA  NY Z662 VIKTIGASTE SKÄL TILL ANSTRÄNGNING I ARBETET NY Z663 NÄST VIKTIGASTE SKÄL TILL ANSTRÄNGNING I ARBETET  NY Z664 SVÅRT ATT ERSÄTTA IP X648 Z665 SVÅRT ATT FÅ NYTT JOBB X649 Z666 KVAR HOS ARBETSGIVARE OM 1 ÅR NY     FÖRVÄRVSARBETANDES FÖRHÅLLANDEN      ARBETSMILJÖ  Z667 TUNGA LYFT X653, Y507, U447, V492, jfr W277 Z668 KROPPSLIGT KRÄVANDE X654, Y508, U448, V493, W278 Z669 DAGLIG SVETTNING X655, Y509, U449, V494, W279 Z670 PSYKISKT ANSTRÄNGANDE X656, Y510, U451, V496, W281 Z671 JÄKT I ARBETET X657, Y511, U452, V497, W282 Z672 ENFORMIGT ARBETE X658, Y512, U453, V498, W283 Z673 PSYKISK PRESS X659, Y513  Z674 PSYKISK AKTIVITET X660, Y514 Z675 STILLASITTANDE ARBETE NY Z676 ARBETE MED ARBETSLEDNING NY Z677 ARBETE MED MÄNNISKOR, EJ ARBETSLEDNING NY Z678 ARBETE MED TEXT OCH/ELLER SIFFROR NY Z679 ARBETE MED FÖREMÅL NY Z680 ARBETE MED HÖG MENTAL ANSTRÄNGNING NY Z681 ARBETE MED LÅG MENTAL ANSTRÄNGNING NY Z682 ARBETE MED KÄNSLOMÄSSIGT KRÄVANDE UPPGIFTER Z683 SJUKFRÅNVARO X669 Z684 BULLER X663, Y515, U456, V501, W286 Z685 ENSIDIGA ARBETSRÖRELSER X664, Y517, U465, V511 Z686 OLÄMPLIGA ARBETSSTÄLLNINGAR X665, Y518, U466, V512 Z687 GAS, DAMM, RÖK X666, Y519, U468, V514, jfr W290 Z688 SMUTSANDE ARBETE X667, jfr U455, jfr V500, jfr W285 Z689 GIFT, SYRA, EXPLOSIVA ÄMNEN X668, Y521, U470, V516, jfr W291 Z690 TILLFREDSSTÄLLELSE MED LÖNEN X671, Y523 Z691 INSTÄLLNING TILL ARBETET NY     FÖRETAGARE/JORDBRUKARE   Z694 NYK80 FÖRETAGARE/JORDBRUKARE Y536, jfr U498 Z695 NYK85 FÖRETAGARE/JORDBRUKARE NY Z700 ANTAL ANSTÄLLDA I FÖRETAG/JORDBRUK jfr X684, jfr Y537,jfr U499,jfr V538, jfr W315 Z701 ARBETSTIMMAR I GENOMSNITT jfr X685, jfr Y542,jfr U501,jfr V540, jfrW317 Z702 TIMMAR HEKTISKA VECKOR jfr X686 Z703 TIMMAR LUGNA VECKOR jfr X687 Z704 ANTAL HEKTISKA VECKOR jfr X688 Z705 SEMESTER 2009 jfr X689,  Z706 SEMESTERVECKOR 2009 jfr X690, jfr Y544,jfr U504, jfr V543, jfr W319 Z707 FÖRETAGETS/JORDBRUKETS VÄRDE NY Z708 ANDEL EGET ÄGANDE I FÖRETAGET/JORDBRUKET NY Z709 PARTNERS ÄGANDE I FÖRETAGET/JORDBRUKET NY     FRILANS / BISYSSLA  Z710 SEI FRILANS/BISYSSLA X691, Y545, jfr U508 Z711 NYK80 FRILANS/BISYSSLA X692, Y546, jfr U509 Z712 NYK85 FRILANS/BISYSSLA X693 Z717 ARBETSTIMMAR I GENOMSNITT X694, Y548, U511, V547, jfr W323     ARBETSLÖSA  Z718 SEI ARBETSLÖSA X695, Y549, jfr U516 Z719 NYK80 ARBETSLÖSA X696, Y550, jfr U517 Z720 NYK85 ARBETSLÖSA X697 Z725 ARBETSLÖSHETSVECKOR  X698, Y551, U525, V559, jfr W345 Z726 ÖNSKAD ARBETSTID NY Z727 INSTÄLLLNING TILL ARBETE NY     PENSIONERADE  Z728 PENSIONSÅR jfr Y552, jfr U552, jfr V584, jfr W362 Z736 FD FÖRETAGARE/JORDBRUKARE – ANSTÄLLDA jfr X705, Y557, U558 Z737 ÖNSKAR ARBETE X706, Y558, U559, V586, W363 Z738 ÖNSKADE ARB TIMMAR PER VECKA X707, Y559, jfr U560-U562 Z739 ORKAR EJ MED ARBETE X708, Y560, U563, V591 Z740 EJ LÖNSAMT Z748 TIMMAR – INKÖP, MATLAGN, DISK X713, Y567, jfr U542, jfr V576 Z749 TIMMAR IP – INKÖP, MATLAGN, DISK X714, Y568, jfr U543, jfr V577 Z750 TIMMAR – KLÄDVÅRD X715, Y569, jfr U536, U538, jfr V570,V572 Z751 TIMMAR IP – KLÄDVÅRD X716, Y570, jfr U537, U539, jfr V571,V573 Z752 TIMMAR – STÄDNING X717, Y571, jfr U540, jfr V574 Z753 TIMMAR IP – STÄDNING X718, Y572, jfr U541, jfr V575 Z754 TIMMAR – REPARATION OCH UNDERHÅLL X719 Z755 TIMMAR IP – REPARATION OCH UNDERHÅLL X720 Z756 TIMMAR – HJÄLP TILL ANHÖRIG ELLER SLÄKTING X721 Z757 TIMMAR – HUSHÅLLSHJÄLP FRÅN EGNA BARN jfr X722 Z758 TIMMAR – HUSHÅLLSHJÄLP FRÅN FÖRÄLDRAR jfr X723 Z759 TIMMAR – HUSHÅLLSHJÄLP FRÅN ANNAN ANHÖRIG/SLÄKTING jfr X724 Z760 TIMMAR – HUSHÅLLSHJÄLP FRÅN KOMMUNAL HEMTJÄNST jfr X725 Z761 TIMMAR – HUSHÅLLSHJÄLP FRÅN PRIVAT KÖPT TJÄNST jfr X726 Z762 TIMMAR – HUSHÅLLSHJÄLP FRÅN ANNAN jfr X727 Z763 FÖRDELNING AV HUSHÅLLETS PENGAR X728 Z764 ÅSIKTSSKILLNAD FÖRDELNING AV HUSHÅLLSARBETE X729 Z765 ÅSIKTSSKILLNAD HUR PENGAR ANVÄNDS X730 Z766 ÅSIKTSSKILLNAD HUR MKT MAKE/MAKA/SAMBO ARBETAR X733 Z767 ÅSIKTSSKILLNAD HUR MKT IP ARBETAR X734 Z768 ANTAL DAGAR IP LÄMNAR BARN X735 Z769 ANTAL DAGAR MAKE/MAKA/SAMBO LÄMNAR BARN X736 Z770 ANTAL DAGAR ANNAN LÄMNAR BARN X737 Z771 ANTAL DAGAR IP HÄMTAR BARN X738 Z772 ANTAL DAGAR MAKE/MAKA/SAMBO HÄMTAR BARN X739 Z773 ANTAL DAGAR ANNAN HÄMTAR BARN X740 Z774 ANTAL DAGAR IP LAGAR MIDDAG X741 Z775 ANTAL DAGAR MAKE/MAKA/SAMBO LAGAR MIDDAG X742 Z776 ANTAL DAGAR ANNAN LAGAR MIDDAG X743 Z777 ANTAL DAGAR IP NATTAR BARN X744 Z778 ANTAL DAGAR MAKE/MAKA/SAMBO NATTAR BARN X745 Z779 ANTAL DAGAR ANNAN NATTAR BARN X746     EKONOMISKA RESURSER    Z780 KONTANTMARGINAL X748, Y574, U576, jfr V602, jfr W375 Z781 ANSKAFFNING AV KONTANTER X749, Y575, U577, jfr V602, jfr W375 Z782 EKONOMISKA SVÅRIGHETER X754, Y584 Z783 FÅR UNDERHÅLL PER MÅNAD X755 Z784 BETALAR UNDERHÅLL PER MÅNAD X756 Z785 ARV NY Z786 TOTALT GIVET EKONOMISKT STÖD, TKR X760 Z787 EKONOMISKT Z788 EKONOMISKT STÖD TILL MOR/FARFÖRÄLDRAR, TKR X762 Z789 EKONOMISKT STÖD TILL BARN, TKR X763 Z790 EKONOMISKT STÖD TILL BARNBARN, TKR X764 Z791 EKONOMISKT STÖD TILL ANNAN PERSON, TKR X765 Z792 TOTALT MOTTAGET EKONOMISKT STÖD, TKR X766 Z793 EKONOMISKT STÖD FRÅN FÖRÄLDRAR, TKR X767 Z794 EKONOMISKT STÖD FRÅN MOR/FARFÖRÄLDRAR, TKR X768 Z795 EKONOMISKT STÖD FRÅN BARN, TKR X769 Z796 EKONOMISKT STÖD FRÅN BARNBARN, TKR X770 Z797 EKONOMISKT STÖD FRÅN ANNAN PERSON, TKR X771 Z798 NÄRA VÄN VID SJUKDOM X775, Y614, U699 Z799 NÄRA VÄN SOM SÄLLSKAP X776, Y615, U700 Z800 NÅGON ATT PRATA MED X777, Y616, U701 Z801 VÄRDE REALA TILLGÅNGAR NY Z802 SKULDER FÖR REALA TILLGÅNGAR NY Z803 REALA TILLGÅNGAR, ÄGANDE NY Z804 FINANSIELLA TILLGÅNGAR NY Z805 FINANSIELLA TILLGÅNGAR, ÄGANDE NY     SÄKERHET TILL LIV OCH EGENDOM    Z806 UTSATT FÖR STÖLD X779, Y585, U593, jfr V618 Z807 UTSATT FÖR SKADEGÖRELSE X780, Y586, U596, V621 Z808 VÅLD MED KROPPSSKADA X781, Y587, U601, V626 Z809 VÅLD UTAN KROPPSSKADA X782, Y588, U602, V627 Z810 ALLVARLIGA HOT X783, Y589, U603, V628     FRITIDSAKTIVITETER    Z811 SEMESTERRESA 2009 X784, Y590, U611, V636, jfr W614 Z812 STUGVISTELSE 2009 X785, Y594, U618, V643, W469 Z813 FISKA ELLER JAGA PÅ FRITIDEN X786, jfr Y596-597, jfr U621-622, jfr V646-647, jfr W472-473 Z814 TRÄDGÅRDSSKÖTSEL X787, Y598, U623, V648, W474 Z815 BIOBESÖK X788, Y599, U624, V649, W475 Z816 TEATERBESÖK X789, Y600, U625, V650, W476 Z817 RESTAURANGBESÖK X790, Y601, U626, V651, W477 Z818 DANS X791, Y602, U627, V652, W478 Z819 BESÖKA SLÄKTINGAR X792, Y604, U632, V657, W483 Z820 HA SLÄKTBESÖK X793, Y605, U633, V658, W485 Z821 BESÖKA VÄNNER OCH BEKANTA X794, Y606, U634, V659, W484 Z822 HA VÄNNER PÅ BESÖK X795, Y607, U635, V660, W486 Z823 BOKLÄSNING X796, Y603, U628, V653, W479 Z824 STUDIECIRKEL X797, Y608, U636, V661, W487 Z825 MUSICERA ELLER SJUNGA I KÖR X798, jfr Y610-611, jfr U638, jfr V663, jfr W488 Z826 HOBBYARBETE Z827 INTERNET NY Z828 MOTIONSAKTIVITET X800, Y613     DELTAGANDE I SAMHÄLLSLIVET    Z829 MEDLEM I FACKFÖRENING X801, Y617, U648,V666, jfr W491 Z830 FACKFÖRBUND X802, Y618, U649, V667, jfr W491 Z831 FACKLIG MÖTESAKTIVITET X803, jfr Y620, jfr U650, jfr V668, jfr W492 Z832 FACKLIGA UPPDRAG X804, Y621, U652, V670, W493 Z833 FACKLIG AKTIVITET X805, Y622, U653, V749, W579 Z834 PARTIMEDLEMSKAP X806, Y623, U654, V671, W494 Z835 POLITISK MÖTESAKTIVITET X807, jfr Y624, jfr U656, jfr V672, jfr W495 Z836 POLITISK AKTIVITET X808, Y625, U658, V750, W578 Z837 VALDELTAGANDE 2006 X809, Y627, U659, V673, jfr W523 Z838 VALDELTAGANDE 2009 EU-PARLAMENTSVALET NY Z839 VALDELTAGANDE 2010, AVSIKT NY Z840 VALDELTAGANDE 2010, FAKTISKT NY Z841 ÖVRIGA FÖRENINGAR X810, Y629, jfr U670, jfr V683, jfr W500 Z842 FÖRENINGSAKTIVITETER X811, Y630 Z843 RELIGIONSTILLHÖRIGHET NY Z844 RELIGIONEN VIKTIG NY Z845 DEMONSTRERAT SENASTE 10 ÅREN NY  Z846 PERSONLIG KONTAKT X815, Y634, U672, V685, W502 Z847 MÖTESTALAT X816, Y635, U673, V686, W503 Z848 SKRIVIT I TIDNING X817, Y636, U674, V687, W504 Z849 POLITISKT DELTAGANDE jfr X818, jfr Y637, jfr U675, jfr V751, jfr W580,W599 Z850 KAN KLAGA X819, Y638, U678, V689, W506 Z851 HJÄLP VID KLAGOMÅL X820, Y639 Z852 TALA ENGELSKA NY Z853 LÄSA ENGELSKA NY     BEDÖMNINGAR    Z854 SAMHÄLLE MED MER PRIVATA ALTERNATIV X824, jfr Y643 Z855 SAMHÄLLE MED SMÅ INKOMSTSKILLNADER X825, Y645 Z856 SAMHÄLLE DÄR MER VÅRD SKER INOM FAMILJEN X826 Z857 SAMHÄLLE MED JÄMSTÄLLDHET INOM FAMILJEN X827 Z858 PROBLEMLÖSARE X828, Y649 Z859 LIVSTILLFREDSSTÄLLELSE X829, Y650 Z860 SVÅRFÖRSTÅELIGT X830, Y651 Z861 STYRA SITT LIV X831, Y652 Z862 IP:S VÄRDERING AV SIN SITUATION X832, Y647, U702 Z863 HAR IP FÅTT DET BÄTTRE ELLER SÄMRE Z864 JAG GILLAR ORDNING OCH REDA NY Z865 JAG VISAR INTE KÄNSLOR INFÖR ANDRA NY Z866 JAG GER INTE UPP VID MOTGÅNGAR NY Z867 JAG BRUKAR VARA LAT NY Z868 JAG TÄNKER OFTA MER PÅ HUR ANDRA HAR DET ÄN PÅ MIG SJÄLV NY Z869 FÖR ATT LYCKAS I LIVET MÅSTE MAN TA RISKER NY Z870 JAG BLIR LÄTT NERVÖS NY Z871 JAG HAR LÄTT FÖR ATT FÅ SAKER GJORDA NY Z872 JAG GILLAR ATT TA ANSVAR NY Z873 DET LÖNAR SIG INTE ATT PLANERA FÖR FRAMTIDEN NY Z874 JAG ÄR BRA PÅ ATT HANTERA STRESS NY Z875 JAG SER ALLTID TILL ATT SLUTFÖRA DET JAG PÅBÖRJAT NY Z876 JAG ÄR EN RISKTAGARE NY Z877 JAG GILLAR INTE ATT ÄNDRA PÅ INVANDA RUTINER NY Z878 JAG BRUKAR LÄGGA NER MÖDA PÅ SAKER SOM BETALAR SIG FÖRST I FRAMTIDEN NY Z879 JAG HAR SVÅRT ATT STÅ EMOT FRESTELSER NY Z880 FRAMGÅNG ÄR ETT RESULTAT AV HÅRT ARBETE SNARARE ÄN TUR NY Z881 JAG HAR EN LIVLIG FANTASI NY Z882 JAG BRUKAR SKJUTA UPP SAKER SOM KÄNNS JOBBIGA NY Z883 VAD SOM HÄNDER MIG BEROR PÅ MINA EGNA HANDLINGAR NY Z884 JAG HAR ETT GOTT SJÄLVFÖRTROENDE NY Z885 JAG HAR INGA KONSTNÄRLIGA INTRESSEN NY Z886 JAG ÄR UTÅTRIKTAD OCH SOCIAL NY Z887 JAG HAR GOD SJÄLVDISCIPLIN NY Z888 JAG GÖR SAKER GRUNDLIGT NY Z889 JAG HÅLLER MIG HELST I BAKGRUNDEN NY Z890 JAG HITTAR OFTA FEL HOS ANDRA NY Z891 JAG ÄR EN NATURLIG LEDARE NY Z892 JAG BRUKAR LITA PÅ FOLK NY Z893 JAG ÄR RESERVERAD NY Z894 JAG ÄR NÄSTAN ALLTID PÅ GOTT HUMÖR NY     REGISTERVARIABLER OCH VIKTER    Z913 NATIONALITET X835, Y654, U705 Z914 LÄN FÖR KYRKOBOKFÖRING X837, Y656, U707 Z915 KOMMUN FÖR KYRKOBOKFÖRING X838, Y657, U708 Z916 H-REGION X841, Y660, jfr U728 Z917 CIVILSTÅND X842, Y661, U715, V814, W704 Z918 MAKE/A FÖDELSEÅR X843, Y663, U716 Z919 INDIVIDVIKT EFTER ORDINARIE DATAINSAMLING OCH BORTFALLSUPPFÖLJNING NY Z920 INDIVIDVIKT EFTER FÖRKORTAD UPPFÖLJNINGSINTERVJU NY Z921 STRATUM 1-3: INDIVIDVIKT EFTER ORDINARIE DATAINSAMLING OCH BORTFALLSUPPFÖLJNING jfr X845, jfr Y666, jfr U727, jfr V797, jfr W942 Z922 STRATUM PZ1 P - DAGENS DATUM X1154    KÖN, ÅLDER, YRKE  PZ2 P – KÖN X1002 PZ3 P – FÖDELSEÅR X1001 PZ4 P - SEI NUVARANDE YRKE NY PZ5 P - NUVARANDE SYSSELSÄTTNING X1044 PZ6 P - NYK80 NUVARANDE YRKE NY PZ7 P - NYK85 NUVARANDE YRKE NY PZ11 P - EGP NUVARANDE YRKE NY    UPPVÄXTFÖRHÅLLANDEN  PZ12 P - BÅDA FÖRÄLDRARNA FÖDDA I SVERIGE X1004 PZ13 P - FÖDELSELAND - FADER X1005 PZ14 P - FÖDELSELAND - MODER X1006 PZ15 P - URPSRUNGSLAND X1008 PZ16 P - IMMIGRATIONSÅLDER X1009    SYSKON, FÖRÄLDRAR OCH UPPVÄXTFAMILJ  PZ17 P - HEL UPPVÄXTFAMILJ X1011 PZ18 P - SEI FADERN X1013 PZ19 P - NYK80 FADERN X1014 PZ20 P - NYK85 FADERN X1015 PZ25 P - SEI MODERN X1017 PZ26 P - NYK80 MODERN X1018 PZ27 P - NYK85 MODERN X1019 PZ32 P - FADERNS UTBILDNING NY PZ33 P - MODERNS UTBILDNING NY    UTBILDNING  PZ34 P - UTBILDNINGSÅR X1023 PZ35 P - HÖGSTA UTBILDNING X1024    SYSSELSÄTTNINGSBIOGRAFI X1025 PZ36 P - ANTAL ÅR I FÖRVÄRVSARBETE X1026 PZ37 P - BÖRJADE ARBETA: ÅRTAL NY PZ38 P - SEI FÖRSTA ARBETE NY PZ39 P - NYK80 FÖRSTA ARBETE NY PZ40 P - NYK85 FÖRSTA ARBETE NY PZ45 P - EGP FÖRSTA ARBETE PZ46 P - SYSSELSÄTTNING VID NUVARANDE SAMMANBOENDES START X1032 PZ47 P – SEI VID NUVARANDE SAMMANBOENDES START X1033 PZ48 P - NYK80 VID NUVARANDE SAMMANBOENDES START X1034 PZ49 P - NYK85 VID NUVARANDE SAMMANBOENDES START X1035 PZ54 P - EGP VID NUVARANDE SAMMANBOENDES START X1030 PZ61 P - I FÖRVÄRVSARBETE SENAST NÄR X1050    PARNTER: ANSTÄLLDAS ARBETSFÖRHÅLLANDEN  PZ62 P - UTBILDNINGSKRAV X1069 PZ63 P - SKOLÅR UTÖVER FOLK-/GRUNDSKOLA X1070 PZ64 P - ARBETSLEDANDE FUNKTION X1071 PZ65 P - ANTAL UNDERSTÄLLDA X1072 PZ66 P - ORDINARIE VECKOARBETSTID X1053 PZ67 P - HUR OFTA ÖVERTID X1055 PZ68 P - ÖVERNATTNING UTANFÖR HEMMET PGA ARBETE: ANTAL NÄTTER/ÅR X1068    PARNTER: LÖN  PZ69 P - MÅNADSLÖN BRUTTO X1073 PZ70 P - MÅNADSLÖN NETTO X1074    PARTNER: HUSHÅLLSARBETE OCH FAMILJ  PZ71 P - TIMMAR P - INKÖP, MATLAGNING, DISK X1097 PZ72 P - TIMMAR MAKE/MAKA/SAMBO - INKÖP, MATLAGNING, DISK X1098 PZ73 P - TIMMAR BARN - INKÖP, MATLAGNING, DISK X1099 PZ74 P - TIMMAR P - KLÄDVÅRD X1100 PZ75 P - TIMMAR MAKE/MAKA/SAMBO - KLÄDVÅRD X1101 PZ76 P - TIMMAR BARN - KLÄDVÅRD X1102 PZ77 P - TIMMAR P - STÄDNING X1103 PZ78 P - TIMMAR MAKE/MAKA/SAMBO - STÄDNING X1104 PZ79 P - TIMMAR BARN - STÄDNING X1105 PZ80 P - TIMMAR P - REPARATION OCH UNDERHÅLL X1106 PZ81 P - TIMMAR MAKE/MAKA/SAMBO - REPARATION OCH UNDERHÅLL X1107 PZ82 P - TIMMAR BARN - REPARATION OCH UNDERHÅLL X1108 PZ83 P - ÅSIKTSSKILLNAD FÖRDELNING AV HUSHÅLLSARBETE X1116 PZ84 P - ÅSIKTSSKILLNAD HUR PENGAR ANVÄNDS X1117 PZ85 P - ÅSIKTSSKILLNAD HUR MKT MAKE/MAKA/SAMBO ARBETAR X1120 PZ86 P - ÅSIKTSSKILLNAD HUR MYCKET P ARBETAR X1119    PARNTER: SYSSELSÄTTNINGSFÖRHÅLLANDEN 2009  PZ87 P - FÖRVÄRVSARBETE 2009: ANTAL VECKOR X1077 PZ88 P - FÖRVÄRVSARB PZ89 P - SAMMANBOENDE TIDIGARE X1079 PZ90 P - ANTAL TIDIGARE SAMMANBOENDEN X1080 PZ91 P - ÅR/MÅNAD SENASTE AVSLUTADE SAMMANBOENDE X1081 PZ92 P - ANTAL EJ HEMMAVARANDE BARN X1082 PZ93 P - ANTAL BORTASMÅBARN X1083 PZ94 P - ANTAL BORTASKOLBARN X1084 PZ95 P - ANTAL BORTA VUXNA BARN X1085 PZ96 P - FÖDELSEÅR BORTABOENDE BARN 1 X1086 PZ97 P - FÖDELSEÅR BORTABOENDE BARN 2 X1087 PZ98 P - FÖDELSEÅR BORTABOENDE BARN 3 X1088 PZ99 P - FÖDELSEÅR BORTABOENDE BARN 4 X1089 PZ100 P - FÖDELSEÅR BORTABOENDE BARN 5 X1090    PARTNER: EKONOMISKA RESURSER  PZ102 P - KONTANTMARGINAL X1122 PZ103 P - ANSKAFFNING AV KONTANTER X1123    PARTNER: HÄLSA  PZ104 P - ALLMÄNT HÄLSOTILLSTÅND X1124 PZ105 P – RÖKER NY PZ106 P - DRICKER VIN, SPRIT MM X1143 PZ107 P - ANTAL GGR DRICKER VIN, SPRIT MM X1144 PZ108 P - ANTAL GLAS X1145 PZ109 P - IP:S VÄRDERING AV SIN SITUATION X1153    PARTNER: ÖVRIGA REGISTERUPPGIFTER  PZ110 P - UTBILDNINGSNIVÅ'

Eftersom alla (och endast) variabler börjar med antingen "Z" eller "PZ" kan jag välja ut dessa och göra till separata strängar på följande vis.

In [29]:
var = [s for s in s1.split() if (s[:2] == 'PZ') or (s[0] == 'Z')]

Här tar jag fram en lista med de variabler som finns i kodbokens innehållsförteckning men som *inte* finns i den DataFrame jag tagit fram ovan.

In [30]:
saknas = [namn for namn in var if namn not in list(LNU.index)]
saknas

['Z1', 'Z21', 'Z45_a', 'Z46_a', 'Z47_a', 'Z830', 'PZ72', 'PZ108']

Z1 finns inte med i någonstans i kodboken annat än i innehållsförteckningen. Samma sak verkar gälla Z21, Z45_a, Z46_a, Z47_a, PZ72 och PZ108. Däremot finns Z830 med i kodboken samtidigt som den saknas i dataframe:n. Förklaringen är att `camelot` inte lyckas läsa in just den sidan. Detta går att se genom att (försöka) läsa in den sida variabeln Z830 finns på (sidan 302).

In [31]:
camelot.read_pdf('LNU302.pdf')

<TableList n=0>

Det blir enklast att manuellt skapa en rad för variabeln Z830.

In [32]:
rad = pd.Series({'Enkät': 'LNU', 'Beskrivning': 'FACKFÖRBUND', 
                 'Itemnr': np.nan,
                'Enkättext': 'Vilket förbund (eller liknande) tillhör du?'}, name = 'Z830')

In [33]:
LNU = LNU.append(rad)

In [34]:
LNU.loc[LNU.index == 'Z830', :]

,Enkät,Beskrivning,Itemnr,Enkättext
Variabelnamn,,,,
Z830,LNU,FACKFÖRBUND,NaN,Vilket förbund (eller liknande) tillhör du?


In [35]:
LNU.sort_index(inplace = True)

# <a id = "6.">6. Skapa .csv</a>

In [37]:
LNU.to_csv('test.csv')